# Final Project

### Import Data to MongoDB

In [1]:
from pymongo import MongoClient
import json
import ast
import pandas as pd
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkConf, SparkContext

In [2]:
# connect to server
client = MongoClient('mongo-csgy-6513-fall.db:27017',
                     username = "dj2145",
                     password = "dj2145", 
                     authSource = "db_dj2145")

mydb = client['db_dj2145']

collection_accidents = mydb.accidents

In [4]:
"""
file_name = ["accidents_2005_to_2007.csv","accidents_2009_to_2011.csv","accidents_2012_to_2014.csv"]
for name in file_name:
    acc_data = pd.read_csv(name)
    payload = json.loads(acc_data.to_json(orient='records'))
    collection_accidents.insert_many(payload)
"""
    

'\nfile_name = ["accidents_2005_to_2007.csv","accidents_2009_to_2011.csv","accidents_2012_to_2014.csv"]\nfor name in file_name:\n    acc_data = pd.read_csv(name)\n    payload = json.loads(acc_data.to_json(orient=\'records\'))\n    collection_accidents.insert_many(payload)\n'

In [3]:
acc_data = pd.read_csv("accidents_2005_to_2007.csv")
acc_data1 = pd.read_csv("accidents_2009_to_2011.csv")
acc_data2 = pd.read_csv("accidents_2012_to_2014.csv")

/opt/conda/envs/bigdata/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (31) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
full = acc_data.append(acc_data1).append(acc_data2)

In [18]:
cities = collection_accidents.find({"Accident_Index":"201201BS70005"}) 
i = 0
for record in cities:
    print(record)
    i += 1
    if i == 10:
        break

In [5]:
acc_data["Did_Police_Officer_Attend_Scene_of_Accident"].value_counts()

Yes    459311
No     108325
Name: Did_Police_Officer_Attend_Scene_of_Accident, dtype: int64

In [6]:
from pyspark.sql import *
from pyspark.sql import Row
import pandas as pd 

from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
import pyspark
from pyspark.sql.functions import hour
from pyspark.sql.functions import col
import pyspark.sql.functions as F


spark = pyspark.sql.SparkSession.builder\
    .master('local[*]')\
    .appName('Dian\'s HW2')\
    .getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/15 06:31:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [27]:

features_to_scale = ["Location_Easting_OSGR","Location_Northing_OSGR","Longitude","Latitude"
                     ,"Number_of_Vehicles","Number_of_Casualties"]

features_strindex = ["Pedestrian_Crossing-Physical_Facilities","Light_Conditions","Weather_Conditions",
                     "Day_of_Week","Speed_limit",
                     "Road_Type",
                     "2nd_Road_Class","2nd_Road_Number",'Pedestrian_Crossing-Human_Control',
                     "Road_Surface_Conditions","Did_Police_Officer_Attend_Scene_of_Accident",
                     "Special_Conditions_at_Site","Carriageway_Hazards"]

acc_data_drop_col = full.drop( columns = ["Accident_Index","Date","Junction_Detail"])

features_full = features_strindex + features_to_scale
acc_no_na = acc_data_drop_col.dropna()


In [28]:
acc_no_na["label"] = acc_no_na["Accident_Severity"].apply(lambda row: 1 if  row == 3 else 0)

/opt/conda/envs/bigdata/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [29]:
acc_no_na["Road_Surface_Conditions"].value_counts()

Dry                          608225
Wet/Damp                     222293
Frost/Ice                     10720
Snow                           3956
Flood (Over 3cm of water)       449
Name: Road_Surface_Conditions, dtype: int64

In [30]:
acc_no_na.columns

Index(['Location_Easting_OSGR', 'Location_Northing_OSGR', 'Longitude',
       'Latitude', 'Police_Force', 'Accident_Severity', 'Number_of_Vehicles',
       'Number_of_Casualties', 'Day_of_Week', 'Time',
       'Local_Authority_(District)', 'Local_Authority_(Highway)',
       '1st_Road_Class', '1st_Road_Number', 'Road_Type', 'Speed_limit',
       'Junction_Control', '2nd_Road_Class', '2nd_Road_Number',
       'Pedestrian_Crossing-Human_Control',
       'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions',
       'Weather_Conditions', 'Road_Surface_Conditions',
       'Special_Conditions_at_Site', 'Carriageway_Hazards',
       'Urban_or_Rural_Area', 'Did_Police_Officer_Attend_Scene_of_Accident',
       'LSOA_of_Accident_Location', 'Year', 'label'],
      dtype='object')

In [32]:
"""
acc_no_na["Accident_Severity"].value_counts()
a1 = acc_no_na[acc_no_na["Accident_Severity"] == 1 ]
a2 = acc_no_na[acc_no_na["Accident_Severity"] == 2 ]
a3 = acc_no_na[acc_no_na["Accident_Severity"] == 3 ]

samplp_num = min(len(a1),len(a2),len(a3))

aa3 = a3.sample(n = samplp_num)
aa1 = a1.sample(n = samplp_num)
aa2 = a2.sample(n = samplp_num)
full_sampeled = aa1.append(aa2).append(aa3)
"""

a1 = acc_no_na[acc_no_na["label"] == 1 ]
a2 = acc_no_na[acc_no_na["label"] == 0 ]

samplp_num = min(len(a1),len(a2))

aa1 = a1.sample(n = 500)
aa2 = a2.sample(n = 500)
full_sampeled = aa1.append(aa2)

spark_df = spark.createDataFrame(full_sampeled)


In [33]:
spark_df.groupBy("label").count().collect()

[Row(label=1, count=500), Row(label=0, count=500)]

In [34]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier,DecisionTreeClassifier

train, test = spark_df.randomSplit(weights=[0.8,0.2], seed=200)


# Random Forest Model

In [35]:
val = [VectorAssembler(inputCols=[f], outputCol=('vec_' + f)) for f in features_to_scale]

qualification_indexer = StringIndexer(inputCol="qualification", outputCol="qualificationIndex")


scale = [StandardScaler(inputCol='vec_' + f, outputCol='scaled_' + f, withMean=True, withStd=True) for f in features_to_scale]

indexer = [StringIndexer(inputCol= f, outputCol= 'indexed_' +f ) for f in features_strindex]

one_hot = [OneHotEncoder( inputCol= 'indexed_' + f  , outputCol='scaled_' + f) for f in features_strindex]

featureArr = [('scaled_' + f) for f in features_full]

va2 = VectorAssembler(inputCols=featureArr, outputCol="features")

#logist_model = LogisticRegression(featuresCol='features',labelCol='Accident_Severity')
#gbt = GBTClassifier(labelCol="Accident_Severity", featuresCol="features", maxIter=10)


rf = RandomForestClassifier(labelCol='label',featuresCol='features', numTrees=10)


stages = val + scale + indexer + one_hot   + [va2,rf]







In [36]:
p = Pipeline(stages=stages)
model = p.fit(train)
#prediction = model.transform(test)

In [38]:
prediction = model.transform(test)

In [39]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(prediction)
print("Test Error = %g" % (1.0 - accuracy))

rfModel = model.stages[2]
print(rfModel)  # summary only

Test Error = 0.507538
VectorAssembler_b2089df87eb1


In [23]:
prediction.select("features", "label", "probability", "prediction").collect()

21/12/15 03:18:14 ERROR Executor: Exception in task 0.0 in stage 228.0 (TID 268)
org.apache.spark.SparkException: Failed to execute user defined function (StringIndexerModel$$Lambda$3373/350742249: (string) => double)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:136)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:759)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$getByteArrayRdd$1(SparkPlan.scala:349)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:898)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartiti

Py4JJavaError: An error occurred while calling o6829.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 228.0 failed 1 times, most recent failure: Lost task 0.0 in stage 228.0 (TID 268) (jupyter-dj2145 executor driver): org.apache.spark.SparkException: Failed to execute user defined function (StringIndexerModel$$Lambda$3373/350742249: (string) => double)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:136)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:759)
	at org.apache.spark.sql.exe

In [17]:
result = prediction.select("features", "label", "probability", "prediction").collect()
correct = 0
total_len = len(full_sampeled)*0.2
for row in result:
    if row.label == row.prediction:
          correct += 1
print("The test accuracy of the model is: ", correct/total_len)

21/12/15 03:16:29 ERROR Executor: Exception in task 1.0 in stage 79.0 (TID 93)
org.apache.spark.SparkException: Failed to execute user defined function (StringIndexerModel$$Lambda$3373/350742249: (string) => double)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:136)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:759)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$getByteArrayRdd$1(SparkPlan.scala:349)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:898)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartition

Py4JJavaError: An error occurred while calling o2330.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 79.0 failed 1 times, most recent failure: Lost task 1.0 in stage 79.0 (TID 93) (jupyter-dj2145 executor driver): org.apache.spark.SparkException: Failed to execute user defined function (StringIndexerModel$$Lambda$3373/350742249: (string) => double)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:136)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:759)
	at org.apache.spark.sql.execut

In [134]:
prediction.groupBy("prediction").count().collect()

21/12/14 03:14:09 WARN TaskSetManager: Stage 1383 contains a task of very large size (28714 KiB). The maximum recommended task size is 1000 KiB.


[Row(prediction=0.0, count=34624), Row(prediction=1.0, count=34634)]

# Logistic Regression Model

In [168]:
val = [VectorAssembler(inputCols=[f], outputCol=('vec_' + f)) for f in features_to_scale]

qualification_indexer = StringIndexer(inputCol="qualification", outputCol="qualificationIndex")


scale = [StandardScaler(inputCol='vec_' + f, outputCol='scaled_' + f, withMean=True, withStd=True) for f in features_to_scale]

indexer = [StringIndexer(inputCol= f, outputCol= 'indexed_' +f ) for f in features_strindex]

one_hot = [OneHotEncoder( inputCol= 'indexed_' + f  , outputCol='scaled_' + f) for f in features_strindex]

featureArr = [('scaled_' + f) for f in features_full]

va2 = VectorAssembler(inputCols=featureArr, outputCol="features")

#logist_model = LogisticRegression(featuresCol='features',labelCol='Accident_Severity')
#gbt = GBTClassifier(labelCol="Accident_Severity", featuresCol="features", maxIter=10)

#rf = RandomForestClassifier(labelCol='Accident_Severity',featuresCol='features', numTrees=50)
lr = LogisticRegression(featuresCol='features',labelCol='label',
                        maxIter=500, regParam=0.01, elasticNetParam=0.1)

#dt = DecisionTreeClassifier(labelCol="Accident_Severity", featuresCol="features")
    
stages = val + scale + indexer + one_hot + [va2,lr]

p = Pipeline(stages=stages)
model_dt = p.fit(train)
prediction_dt = model_dt.transform(test)



21/12/14 03:40:33 WARN TaskSetManager: Stage 1752 contains a task of very large size (18241 KiB). The maximum recommended task size is 1000 KiB.
21/12/14 03:40:36 WARN TaskSetManager: Stage 1755 contains a task of very large size (18241 KiB). The maximum recommended task size is 1000 KiB.
21/12/14 03:40:39 WARN TaskSetManager: Stage 1758 contains a task of very large size (18241 KiB). The maximum recommended task size is 1000 KiB.
21/12/14 03:40:42 WARN TaskSetManager: Stage 1761 contains a task of very large size (18241 KiB). The maximum recommended task size is 1000 KiB.
21/12/14 03:40:45 WARN TaskSetManager: Stage 1764 contains a task of very large size (18241 KiB). The maximum recommended task size is 1000 KiB.
21/12/14 03:40:48 WARN TaskSetManager: Stage 1767 contains a task of very large size (18241 KiB). The maximum recommended task size is 1000 KiB.
21/12/14 03:40:53 WARN TaskSetManager: Stage 1770 contains a task of very large size (18241 KiB). The maximum recommended task siz

In [169]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(prediction_dt)
print("Test Error = %g" % (1.0 - accuracy))



21/12/14 03:41:44 WARN TaskSetManager: Stage 1842 contains a task of very large size (18241 KiB). The maximum recommended task size is 1000 KiB.


Test Error = 0.40365


In [170]:
prediction_dt.groupBy("prediction").count().collect()

21/12/14 03:42:01 WARN TaskSetManager: Stage 1844 contains a task of very large size (18241 KiB). The maximum recommended task size is 1000 KiB.


[Row(prediction=0.0, count=25203), Row(prediction=1.0, count=17926)]

In [171]:
result = prediction_dt.select("features", "label", "probability", "prediction").collect()
correct = 0
total_len = len(full_sampeled)*0.2
for row in result:
    if row.label == row.prediction:
          correct += 1
print("The test accuracy of the model is: ", correct/total_len)

21/12/14 03:42:14 WARN TaskSetManager: Stage 1847 contains a task of very large size (18241 KiB). The maximum recommended task size is 1000 KiB.


The test accuracy of the model is:  0.5951774887767853


# Regression Model